In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import unicodedata
import json
import os
import concurrent.futures
import time



In [5]:
with open('../letterboxd_proj_data/failed_fetches.jsonl', 'r') as json_file:
    json_list = list(json_file)
print(len(json_list))
# for json_str in json_list[:10]:
#     result = json.loads(json_str)
#     print(f"result: {result}")

11322


In [6]:
with open('../letterboxd_proj_data/usernames_sample_100.jsonl', 'r') as json_file:
    json_list = list(json_file)
print(len(json_list))

182284


In [4]:
# Set headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
}

- 30/35 responses per page, popular reviews section double counts the top 5 people
- 1.5-2s per response

In [ ]:
# failed_fetch = 0
# usernames = set()
# for i in range(1):
#     url = "https://letterboxd.com/members/popular/this/year/page/{}/".format(i)

#     # Fetch the webpage
#     response = requests.get(url, headers=headers)
#     # Check if request was successful
#     if response.status_code == 200:
#         # Parse HTML
#         soup = BeautifulSoup(response.text, "html.parser")
#         user_elements = soup.select(".person-summary .name")

#         # Extract and print usernames
#         for user in user_elements:
#             usernames.add(user["href"][1:-1])
#     else:
#         print("failed iter {}", i)
#         failed_fetch += 1

In [ ]:
with open('../letterboxd_proj_data/kaggle_data/usernames.json', 'r') as file:
    try:
        usernames_json = json.load(file)
    except json.JSONDecodeError:
                usernames_json = []

In [16]:
usernames_sample = usernames_json['usernames'][:1]

## Scrape review data
- scraping 1 users reviews took roughly 1 min
- url doesnt work past 256

In [1]:
def scrape_single_review_page(user, i, session, output_file_name):
    url = "https://letterboxd.com/{}/films/reviews/page/{}".format(user, i)
    response = session.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        # Find all film detail blocks
        film_blocks = soup.find_all("div", class_="film-detail-content")
        # print(user, url)
        # Iterate over each block and extract the required information
        for film in film_blocks:
            # Extract film title
            film_title_tag = film.find("h2", class_="headline-2 prettify").find("a")
            film_title = film_title_tag.text.strip() if film_title_tag else "Unknown Title"
            # print(film_title)


            # Extract star rating
            rating_tag = film.find("span", class_="rating")
            rating_text = rating_tag.text.strip() if rating_tag else None
            if rating_text:
                rating = rating_text.count('\u00BD')*0.5 + rating_text.count('\u2605')

            # Extract date
            date_tag = film.find("span", class_="_nobr")
            date_string = date_tag.text.strip() if date_tag else None
            if date_string:
                date_object = datetime.strptime(date_string, "%d %b %Y")
                unix_timestamp = int(date_object.timestamp())
            else:
                print("no date ", user, film_title, date_tag)

            # Check if "icon-liked" exists in the attribution block
            liked_icon = film.find("span", class_="has-icon icon-16 icon-liked")
            liked = 1 if liked_icon else 0

            # Extract review text
            review_body_tag = film.find("div", class_="body-text -prose js-review-body js-collapsible-text")
            review_text = review_body_tag.get_text(" ", strip=True) if review_body_tag else None

            # save review to json
            reviews_file_name = "../letterboxd_proj_data/{}.json".format(output_file_name)
            save_review_to_json(reviews_file_name, user, film_title, unix_timestamp, rating, liked, review_text)
    else:
        log_failed_fetch((i, user))
    # time.sleep(random.uniform(0.5, 1.5))

def save_review_to_json(file_path, user, film_title, unix_timestamp, rating, liked, review_text):
    # Create a dictionary with the extracted details
    review_data = {
        "user": user,
        "title": film_title,
        "review_date": unix_timestamp,
        "rating": rating,
        "liked": liked,
        "review_text": review_text
    }
    
    # Load existing data if the file exists, else start with an empty list
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            try:
                data = json.load(file)
            except json.JSONDecodeError:
                data = []  # If the file is empty or corrupted, reset to an empty list
    else:
        data = []
    
    # Append new data to the list
    data.append(review_data)

    # Save the updated data back to the file
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)



def log_failed_fetch(i, user):
    print('fail L nerd moment')
    # Load existing data if the file exists, else start with an empty list
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            try:
                data = json.load(file)
            except json.JSONDecodeError:
                data = []  # If the file is empty or corrupted, reset to an empty list
    else:
        data = []
    
    # Append new data to the list
    data.append((i,user))

    # Save the updated data back to the file
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)


In [ ]:
def get_reviews(usernames, output_file_name):
    # Create a session
    session = requests.Session()
    for user in usernames:
        url = "https://letterboxd.com/{}/films/reviews/".format(user)
        
        # Fetch the webpage
        response = session.get(url, headers=headers)
        # Check if request was successful
        if response.status_code == 200:
            # Parse HTML
            soup = BeautifulSoup(response.text, "html.parser")
            last_page = max(int(a.get_text()) for a in soup.select(".paginate-page a") if a.get_text().isdigit())

            for i in range(1, last_page + 1):
                scrape_single_review_page(user, i, session, output_file_name)
        else:
            log_failed_fetch((-1, user))
    session.close()


In [ ]:
get_reviews(usernames_sample, "usernames_sample_1") # took 3.30 with just session improvement

no date  deathproof 13th <span class="_nobr"><time class="localtime-d-mmm-yyyy" datetime="2020-06-05T03:36:38.905Z"></time></span>


In [ ]:
MAX_THREADS = 30

In [29]:

def get_reviews_multi(usernames, output_file_name):
    # Create a session
    session = requests.Session()
    for user in usernames:
        url = "https://letterboxd.com/{}/films/reviews/".format(user)
        
        # Fetch the webpage
        response = session.get(url, headers=headers)
        # Check if request was successful
        if response.status_code == 200:
            # Parse HTML
            soup = BeautifulSoup(response.text, "html.parser")
            last_page = max(int(a.get_text()) for a in soup.select(".paginate-page a") if a.get_text().isdigit())
            threads = min(MAX_THREADS, last_page)
            
            with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
                futures = [
                    executor.submit(scrape_single_review_page, user, i, session, output_file_name)
                    for i in range(1, last_page + 1)
                ]
                # Wait for all tasks to complete
                concurrent.futures.wait(futures)

        else:
            log_failed_fetch((-1, user))
    session.close()


In [32]:
get_reviews_multi(usernames_sample, "usernames_sample_4") # took 30s with multi

no date  deathproof 13th <span class="_nobr"><time class="localtime-d-mmm-yyyy" datetime="2020-06-05T03:36:38.905Z"></time></span>
